In [1]:
from __future__ import print_function

import os
import sys


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_context('talk')
import random
from scipy.stats import randint
import re
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

PROJ_ROOT = os.path.join(os.pardir)

In [2]:
#Create the path to the data and read into a pandas dataframe

terry_data = os.path.join(PROJ_ROOT, 
                         'data', 'processed',
                         'Terry_Stops_Clean.csv')

data = pd.read_csv(terry_data, parse_dates = ['date'], 
                   index_col = 'date', dtype = {'officer_race':'category','officer_gender':'category',
                                                'subject_age':'category','subject_race':'category',
                                                'subject_gender': 'category','stop_resolution': 'category',
                                                'weapon_type':'category','initial_call_type':'category',
                                                'call_type':'category','arrest':'int32', 'frisk':'float',
                                                'precinct':'category', 'sector':'category', 'beat': 'category'})

data.sort_index(inplace = True)

In [3]:
data.head()

,officer_id,officer_age,officer_race,officer_gender,officer_squad,subject_id,subject_age,subject_race,subject_gender,stop_resolution,weapon_type,initial_call_type,call_type,arrest,frisk,precinct,sector,beat
date,,,,,,,,,,,,,,,,,,
2015-03-16,1757,31.0,White,M,NaN,1432.0,18 - 25,Black,Male,Arrest with GO or Supplemental,None,NaN,NaN,1,0.0,NaN,NaN,NaN
2015-03-16,1735,38.0,White,M,NaN,22667.0,18 - 25,White,Male,Street Check,None,NaN,NaN,0,0.0,NaN,NaN,NaN
2015-03-16,1735,38.0,White,M,NaN,20151.0,36 - 45,Multi-Racial,Male,Street Check,None,NaN,NaN,0,0.0,NaN,NaN,NaN
2015-03-17,1735,38.0,White,M,NaN,10743.0,26 - 35,White,Male,Street Check,None,NaN,NaN,0,0.0,NaN,NaN,NaN
2015-03-17,1561,31.0,White,M,NaN,NaN,26 - 35,Black,Male,GO Report,None,THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),911,0,0.0,East,G,G3


In [4]:
def split_mean(x):
    #Function to split the Age bins and return the mean of the two numbers
        if '-' in x:
            split_list = x.split('-')
            mean = (float(split_list[0]) + float(split_list[1]))/2
        else:
            mean = 56
        return mean

In [5]:
ofc_columns = ['subject_race','officer_id', 'beat', 'call_type', 'initial_call_type', 'officer_race',
                'frisk', 'sector']

df = data[ofc_columns]

    
# view only stops when the officer initiates the stop
df = df[df.call_type == 'ONVIEW']


#remove ambiguous subject_races
df = df[(df.subject_race != 'Unknown') & (df.subject_race != 'Other')]

#subset the data to include the initial call types that prompted the stop, limit to 5 or more to capture any trends
ls = list(df.initial_call_type.value_counts()[df.initial_call_type.value_counts() >= 5].index)
df = df[df.initial_call_type.isin(ls)]

#subset the data to include officers with at least 15 stops or more
ls = list(df.officer_id.value_counts()[df.officer_id.value_counts() >= 10].index)
df = df[df.officer_id.isin(ls)]
df.head()

df.drop(['call_type'], axis = 1, inplace = True) #drop the call type feature

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2905 entries, 2015-03-18 to 2019-05-06
Data columns (total 7 columns):
subject_race         2905 non-null category
officer_id           2905 non-null int64
beat                 2905 non-null category
initial_call_type    2905 non-null category
officer_race         2905 non-null category
frisk                2905 non-null float64
sector               2904 non-null category
dtypes: category(5), float64(1), int64(1)
memory usage: 95.8 KB


In [6]:

#create the pattern to match the beat entries
pattern = re.compile( '^[A-Z][1-9]$')

#drop any NaNs in the stop_resolution column
#df = df.dropna(subset = ['stop_resolution'])

#use the pattern created to subset the data and get rid of the erroneous entries
df = df[(df.beat.str.contains(pattern))]

#remove all unused categories
for col in df.select_dtypes(include = ['category']).columns:
   df[col] = df[col].cat.remove_unused_categories()

    #capture the category codes & corresponding strings for the 'race' classes
race_cat_codes = dict(enumerate(df['subject_race'].cat.categories))

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2889 entries, 2015-03-18 to 2019-05-06
Data columns (total 7 columns):
subject_race         2889 non-null category
officer_id           2889 non-null int64
beat                 2889 non-null category
initial_call_type    2889 non-null category
officer_race         2889 non-null category
frisk                2889 non-null float64
sector               2889 non-null category
dtypes: category(5), float64(1), int64(1)
memory usage: 89.0 KB


In [7]:
df.head()

,subject_race,officer_id,beat,initial_call_type,officer_race,frisk,sector
date,,,,,,,
2015-03-18,American Indian / Alaskan Native,1827,C2,TRAFFIC STOP - OFFICER INITIATED ONVIEW,White,0.0,C
2015-03-18,White,1735,E2,WARRANT - FELONY PICKUP,White,0.0,E
2015-03-18,White,1735,E2,WARRANT - FELONY PICKUP,White,0.0,E
2015-03-18,Black,1735,E2,WARRANT - FELONY PICKUP,White,0.0,E
2015-03-18,White,1735,E2,WARRANT - FELONY PICKUP,White,0.0,E


In [8]:
#import relevent classifiers
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#import preprocessing, metrics & pipelines
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, roc_auc_score


/Users/nilanlovelace/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [9]:
#The metric that will be used is log loss. log loss is a log function is a measure of error. 
#The error should be as small as possible.

def compute_log_loss(predicted, actual, eps = 1e-14):
    #computes the logarithmic loss between predicted and actual when these are 1d arrays
    predicted = np.clip(predicted, eps, 1-eps)
    loss = -1 * np.mean(actual * np.log(predicted)
                       + (1 - actual)
                       * np.log(1-predicted))
    return loss

def consolidate_array(arr, cols = [0,1,2,3,4,5]):
    #function to transform the dummies array to a single column
    
    df = pd.DataFrame(arr, columns = cols)
    return(df.idxmax(axis = 1).values)

In [10]:

#set up the target variable
y = pd.get_dummies(df['subject_race']).values


#set the x variables by converting the categorical text features to dummy variables
X = pd.get_dummies(df.reset_index(drop = True).drop(['subject_race'], axis = 1),
                          columns = ['officer_id', 'beat', 'initial_call_type', 'officer_race',
                                     'sector', 'frisk'] )

# 'officer_race', 'officer_age', 'officer_gender', 'weapon_type', 'arrest','precinct','sector'
    
#    'subject_gender', 'subject_race', 'beat', 'officer_squad','stop_resolution','subject_id', 'call_type'

In [11]:
print(y.shape)
print(X.shape)

(2889, 6)
(2889, 277)


In [12]:
#split the data to test & training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 5, stratify = y)


#build the pipline with upsampling & scaling the data
pipeline = imbPipeline([('sm', SMOTE(random_state = 5, sampling_strategy = 'not majority')),
                        ('scale', StandardScaler()),
                        ('clf', OneVsRestClassifier(LogisticRegression()))
                       ])

#paramters for tuning
parameters = [
    {'clf' : [ OneVsRestClassifier(LogisticRegression(random_state = 5))],
    'clf__estimator__C' : np.logspace(-5, 8, 10),
    'clf__estimator__solver' : ['lbfgs']},
    {'clf' : [ OneVsRestClassifier(RandomForestClassifier(random_state = 5))],
    'clf__estimator__max_depth':[4, 3, None],
     'clf__estimator__criterion': ['gini'],
    'clf__estimator__n_estimators' : [10,25,50,100],
    'clf__estimator__max_features' : ['auto', 2, 4]},
]

#create the grid search object

cv = GridSearchCV(pipeline, param_grid = parameters,
                    scoring = 'neg_log_loss', 
                    refit = True, 
                    cv = 5, 
                    verbose= True, 
                    n_jobs = -1)


#fit the data
cv.fit(X_train, y_train)

Fitting 5 folds for each of 46 candidates, totalling 230 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 230 out of 230 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('sm',
                                        SMOTE(k_neighbors=5, kind='deprecated',
                                              m_neighbors='deprecated',
                                              n_jobs=1, out_step='deprecated',
                                              random_state=5, ratio=None,
                                              sampling_strategy='not majority',
                                              svm_estimator='deprecated')),
                                       ('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        One...
                                            

In [13]:
fname = 'predict_race_model.sav'
pickle.dump(cv, open(fname, 'wb'))

In [14]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('sm',
                 SMOTE(k_neighbors=5, kind='deprecated',
                       m_neighbors='deprecated', n_jobs=1,
                       out_step='deprecated', random_state=5, ratio=None,
                       sampling_strategy='not majority',
                       svm_estimator='deprecated')),
                ('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=0.21544346900318823,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                       

In [15]:
#predict on the test set
y_pred = cv.predict(X_test)

y_predict_proba = cv.predict_proba(X_test)

y_t = consolidate_array(y_test)
y_p = consolidate_array(y_pred)

In [16]:
#accuracy score
acc_score = cv.score(X_test, y_test)
log_loss = compute_log_loss(y_predict_proba, y_test)
print('The accuracy score is: {}'.format(acc_score))
print('The Log Loss score is: {}'.format(log_loss))


#print the confusion matrix and classification report from the best model
print(confusion_matrix(y_t, y_p))
print(classification_report(y_t, y_p))

The accuracy score is: -1.35910674553924
The Log Loss score is: 0.3615132892829312
[[ 19   0   0   0   0   0]
 [ 16   0   1   0   0   2]
 [163   0   4   0   0  12]
 [ 32   0   1   0   0   1]
 [ 18   0   0   0   0   0]
 [261   0   7   0   0  41]]
              precision    recall  f1-score   support

           0       0.04      1.00      0.07        19
           1       0.00      0.00      0.00        19
           2       0.31      0.02      0.04       179
           3       0.00      0.00      0.00        34
           4       0.00      0.00      0.00        18
           5       0.73      0.13      0.22       309

    accuracy                           0.11       578
   macro avg       0.18      0.19      0.06       578
weighted avg       0.49      0.11      0.14       578



/Users/nilanlovelace/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
#print the log loss for each column
for col in np.arange(0,6):
    print( compute_log_loss(y_predict_proba[:,col], y_test[:,col]), '\t',race_cat_codes[col])

0.16041256484356467 	 American Indian / Alaskan Native
0.19484769861610518 	 Asian
0.6477534210553716 	 Black
0.24609256444916072 	 Hispanic
0.18335560002078105 	 Multi-Racial
0.7366178867126035 	 White


In [18]:
pd.concat([pd.DataFrame(y_predict_proba), pd.DataFrame(y_test)], axis = 1).head(10)

,0,1,2,3,4,5,0,1,2,3,4,5
0,0.099866,0.115557,0.229073,0.138338,0.133879,0.373696,0,0,0,0,0,1
1,0.085799,0.113874,0.254760,0.142988,0.147882,0.360705,0,0,0,0,0,1
2,0.088878,0.115984,0.241296,0.151114,0.130483,0.364667,0,0,0,0,0,1
3,0.099761,0.111847,0.238745,0.131556,0.131440,0.350686,0,0,1,0,0,0
4,0.092617,0.108494,0.256628,0.140611,0.136821,0.370298,0,0,1,0,0,0
5,0.089149,0.119316,0.254578,0.134699,0.140909,0.361363,0,0,1,0,0,0
6,0.090110,0.105764,0.246246,0.141092,0.139451,0.365711,0,0,0,0,0,1
7,0.083010,0.098535,0.251613,0.141454,0.129481,0.331679,0,0,1,0,0,0
8,0.090502,0.127177,0.240248,0.139017,0.137481,0.365303,0,0,1,0,0,0
9,0.091044,0.121329,0.256539,0.139222,0.132071,0.356938,0,0,1,0,0,0


In [19]:

#obtain the feature importances from within the GridSearchCV, pipeline, & OneVsRest objects
feat_imp = [x.coef_ for x in cv.best_estimator_.steps[2][1].estimators_]
feat_imp = np.mean(feat_imp, axis = 0)

#place the feature importances in a dataframe
feature_importances = pd.DataFrame(feat_imp[0],
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance',ascending=True)
feature_importances.head(3)

,importance
initial_call_type_AUTO RECOVERY,-0.385033
officer_id_1636,-0.318069
officer_id_2094,-0.311826


## Findings

**The model performance metrics:**
 - Log Loss .36
 - Precision .52
 - Recall .13
 - F1 Score .15
    
    The model is a small improvement from the previous model increasing Recall & the F1 Score, though is still not making predictions confidently enough to improve the F1 score to an acceptable level. We can see from predicted probabilities, the predictions are a bit more confident than the previous model. 
    The top 3 features are 'beat' locations and negative, suggesting that subjects stopped in these neighborhoods have a slightly higher probability of *not* being frisked.

A Stop does not necessarily lead to a 'frisk', another set of behaviors observed by an experienced officer needs to be evaluated before a frisk is legitimately performed.